In [ ]:

object SchemaManager {

  // ---------- Schema operations ----------

    // create all DAP schemas
    def createDapSchemas(): Unit = {

      SchemaResolver.DAP_SCHEMAS.foreach { schema => 
        createSchema(schema) 
      }
    }
    
    def createSchema(
      schemaName: String
    ): Unit = {

      spark.sql(s"CREATE SCHEMA IF NOT EXISTS $schemaName")
    }

    // drop all DAP tabls & schema - DANGER!
    // removed for safty

    def purgeSchema(
      schemaName: String,
      cascade: Boolean = false
    ): Unit = {

      val cascadeSql = if (cascade) "CASCADE" else ""
      spark.sql(s"DROP SCHEMA IF EXISTS $schemaName $cascadeSql")
    }


    // ---------- Table operations ----------

    // enable CDF for all DAP tables
    def enableCdfDapTables(): Unit = {

      SchemaResolver.DAP_SCHEMAS.foreach { schema =>

        val tableList = spark.sql(s"SHOW TABLES IN $schema")
          .filter("isTemporary = false") 

        tableList.collect().map { row =>
          val tableName = row.getAs[String]("tableName")
          val fullTablePath = s"${schema}.$tableName"
          enableCdf(fullTablePath)
        }
      }
    }

    def enableCdf(
      tableName: String
    ): Unit = {
      if (spark.catalog.tableExists(tableName)) {
        spark.sql(
          s""" 
            |ALTER TABLE $tableName
            |SET TBLPROPERTIES (delta.enableChangeDataFeed = true);
          """.stripMargin
        )
      }
      else {
        println(s"The table or view `$tableName` cannot be found")
      }

    }

    // optimize All DAP tables 
    def optimizeDapTables(
      zorderMap: Map[String, Seq[String]] = Map.empty
    ): Unit = {

      SchemaResolver.DAP_SCHEMAS.foreach { schema =>

        val tableList = spark.sql(s"SHOW TABLES IN $schema")
          .filter("isTemporary = false")

        tableList.collect().foreach { row =>
          val tableName = row.getAs[String]("tableName")
          val fullTablePath = s"${schema}.$tableName"
          // Safely get Z-ORDER columns (empty if not found)
          val zorderColsForTable: Seq[String] =
            zorderMap.getOrElse(tableName, Seq.empty)

          optimizeTable(
            tableName = fullTablePath,
            zorderCols = zorderColsForTable
          )
        }
      }
    }


    def optimizeTable(
          tableName: String,
          zorderCols: Seq[String] = Seq.empty
      ): Unit = {
        val sql =
          if (zorderCols.nonEmpty)
            s"OPTIMIZE $tableName ZORDER BY (${zorderCols.mkString(", ")})"
          else
            s"OPTIMIZE $tableName"

        spark.sql(sql)
    }


  // sourceSchema, targetSchema - full path for  catalog, schema and version
  // e.g. ag_ra_search_analytics_data_dev.dap_reference_v_1_0_1
  def cloneDapSchemas(
      sourceSchemaFull: String,
      targetSchemaFull: String,
  ): Unit = {

    println(
      s"Cloning schema: $sourceSchemaFull -> $targetSchemaFull"
    )

    // Ensure target schema exists
    spark.sql(s"CREATE SCHEMA IF NOT EXISTS $targetSchemaFull")

    // Get table list from source schema
    val tables = spark
      .sql(s"SHOW TABLES IN $sourceSchemaFull")
      .select("tableName")
      .collect()
      .map(_.getString(0))

    // Clone each table
    tables.foreach { table =>
      val stmt =
        s"""
          |CREATE TABLE IF NOT EXISTS $targetSchemaFull.$table
          |DEEP CLONE $sourceSchemaFull.$table
          |""".stripMargin

      println(stmt)
      spark.sql(stmt)
    }
  }

}